In [1]:
# if (!require("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")
# BiocManager::install(version = "3.10")
# BiocManager::install("randomForest")

In [2]:
setwd('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.2. Smoking Status Prediction/Input')
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.1. Smoking Status Prediction/Output')
cur_date = "021922"

library(tidyverse)
library(caret)
library(e1071)
library(Hmisc)
library(glmnet)
library(pROC)
library(randomForest)

#reading in file
molecular_feature_df_w_Ecig_40 = read_csv("20220217_mf_qrilc_w40.csv") %>%
     rename(Smoking_Status = sample)
molecular_feature_df_wo_Ecig_40 = read_csv("20220217_mf_qrilc_wo40.csv") %>%
     rename(Smoking_Status = sample)
# ("021422_imputed_cytokine_data_w_Ecig40.csv") %>%
#     rename(Smoking_Status = X1)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: survival


Attaching package: ‘survival’


The following object is masked from ‘package:caret’:

    cluster


Loading required package: Formula


Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required package: Matrix


Attaching package: ‘Mat

In [3]:
head(molecular_feature_df_w_Ecig_40)

Smoking_Status,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,⋯,MF_62853,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NS_070,19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,⋯,19.06974,19.10260,17.17255,16.92465,17.09794,19.06203,19.13556,16.93182,17.55702,17.05593
NS_110,21.08622,22.58769,21.05378,17.93584,17.33098,20.41479,20.37485,21.12876,21.01488,⋯,18.02560,17.82667,19.37426,16.23450,19.06421,18.06379,19.39103,16.72112,19.27343,19.30881
NS_143,13.12099,14.44659,13.46827,16.51009,14.93820,15.66299,15.72731,13.78698,15.18975,⋯,19.14095,19.02411,19.56175,17.17120,16.97302,19.49121,17.09470,15.56214,19.55038,19.54912
NS_152,21.60361,23.22761,21.99010,16.22547,17.59020,21.53908,21.14531,22.07857,21.97502,⋯,19.28751,19.54886,16.78848,16.46750,19.47635,19.44477,16.99617,16.09531,19.28810,19.32788
NS_187,19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,⋯,16.70042,17.85969,18.97406,18.18813,18.50336,19.19085,19.41757,16.88993,19.35678,18.95550
NS_196,22.53459,24.36662,22.77595,16.69956,16.38411,22.55912,17.30008,22.80146,22.09338,⋯,16.86999,15.82637,19.46467,16.97149,17.46699,16.32598,15.62938,17.37207,15.84334,17.82000


In [4]:
#first making smoking status column binary denoted by 0 = non smoker and 1 = ecig user
molecular_feature_df_w_Ecig_40 = molecular_feature_df_w_Ecig_40 %>%
    #grepl is synonymous with "contains"
    #necessary to put it into factor and relevel, so algorithm understands which data is considered baseline
    mutate(Smoking_Status = relevel(factor(ifelse(grepl("Ecig", Smoking_Status), 1, 0)), ref = "0"))

molecular_feature_df_wo_Ecig_40 = molecular_feature_df_wo_Ecig_40 %>%
    #grepl is synonymous with "contains"
    #necessary to put it into factor and relevel, so algorithm understands which data is considered baseline
    mutate(Smoking_Status = relevel(factor(ifelse(grepl("Ecig", Smoking_Status), 1, 0)), ref = "0"))

head(molecular_feature_df_w_Ecig_40)

Smoking_Status,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,⋯,MF_62853,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,⋯,19.06974,19.10260,17.17255,16.92465,17.09794,19.06203,19.13556,16.93182,17.55702,17.05593
0,21.08622,22.58769,21.05378,17.93584,17.33098,20.41479,20.37485,21.12876,21.01488,⋯,18.02560,17.82667,19.37426,16.23450,19.06421,18.06379,19.39103,16.72112,19.27343,19.30881
0,13.12099,14.44659,13.46827,16.51009,14.93820,15.66299,15.72731,13.78698,15.18975,⋯,19.14095,19.02411,19.56175,17.17120,16.97302,19.49121,17.09470,15.56214,19.55038,19.54912
0,21.60361,23.22761,21.99010,16.22547,17.59020,21.53908,21.14531,22.07857,21.97502,⋯,19.28751,19.54886,16.78848,16.46750,19.47635,19.44477,16.99617,16.09531,19.28810,19.32788
0,19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,⋯,16.70042,17.85969,18.97406,18.18813,18.50336,19.19085,19.41757,16.88993,19.35678,18.95550
0,22.53459,24.36662,22.77595,16.69956,16.38411,22.55912,17.30008,22.80146,22.09338,⋯,16.86999,15.82637,19.46467,16.97149,17.46699,16.32598,15.62938,17.37207,15.84334,17.82000


The goal of this analysis is to use molecular features [continuous] to predict smoking staus (dichotomous). This will be done using Support Vector Machine (SVM) based on a logistic regression model for classification and Random Forest.

# Random Forest

- an ensemble learning method operating by constructing a multitude of decision trees at training time, which uses multiple methods to obtain a better predictive performance and includes bagging and random forest
- algorithm uses a bootstrop aggregation, to reduce overfitting the training datset but only a subset of the features are used hence decorrelation of predictors

In [5]:
rf_classification = function(dataset, outcome, pred_outcome){
    #setting for reproducibility
    set.seed(12)
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV

    metrics = data.frame()
    variable_importance_df = data.frame()
    
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        ntree_values = c(50, 250, 500) #number of trees (not sure if we wanted something different)
        p = dim(dataset)[2] - 1 #number of variables in dataset
        mtry_values = c(p/2, sqrt(p), p)

        #will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        #finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors <- which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))

        reg_rf <- randomForest(as.formula(paste0(outcome, "~.")), data = data_train,
        #need to go back and figure out how the number of trees/ variables make a difference even though
        #the errors are the same!!!!
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)], 
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])

        #predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")


        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        #calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        #extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
        
        #extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "MF")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        #adding values to df
        metrics = rbind(metrics, matrix_values)
    }
    
    #taking averages/sd 
    metrics = metrics %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(MF) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        #sorting by most important variables
        arrange(-MeanDecreaseGini)
  
    #return training set, matrix and variable importance values
    return(list(data_train, metrics, variable_importance_df))

}

#calling fn
rf_values_w_Ecig_40 = rf_classification(molecular_feature_df_w_Ecig_40, "Smoking_Status", "pred_Smoking_Status")
rf_values_wo_Ecig_40 = rf_classification(molecular_feature_df_wo_Ecig_40, "Smoking_Status", "pred_Smoking_Status")

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases



In [7]:
#viewing results
rf_confusion_matrix = data.frame(Model = c(rep(c("Original RF"), times = 2)), Dataset = c("Ecig_40", "no_Ecig_40"),
                                 Kernel = c(rep(c(NA), times = 2)), rbind(rf_values_w_Ecig_40[[2]], 
                                                                          rf_values_wo_Ecig_40[[2]]))
rf_confusion_matrix

#viewing most significant features
head(rf_values_w_Ecig_40[[3]])
head(rf_values_wo_Ecig_40[[3]]) 

Model,Dataset,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Original RF,Ecig_40,0.9047619,0.8428571,0.9666667,0.9600000,0.9047619
Original RF,no_Ecig_40,0.9047619,0.8428571,0.9666667,0.9666667,0.9047619


MF,MeanDecreaseGini
<chr>,<dbl>
MF_97,7.05424062
MF_5,4.68378298
MF_292,0.27705811
MF_3345,0.24800949
MF_1713,0.18760701
MF_4803,0.16542196
MF_7423,0.15562403
MF_5782,0.15079241
MF_7221,0.14144676


MF,MeanDecreaseGini
<chr>,<dbl>
MF_97,7.05972731
MF_5,4.08153303
MF_13899,0.29520675
MF_4858,0.19183248
MF_5547,0.19155348
MF_292,0.18847797
MF_17380,0.15940730
MF_108,0.15200375
MF_873,0.15093885


Rerunning RF model with noise variables to determine, which predictors are significant above the background noise. 

In [10]:
train_vars_noise_w_Ecig_40 = rf_values_w_Ecig_40[[1]]
train_vars_noise_wo_Ecig_40 = rf_values_wo_Ecig_40[[1]]

# Add random noise predictors as an additional method to evaluate model performance
# Adding a column that contains randomly shuffled values from one of the molecules; sampling with replacement
train_vars_noise_w_Ecig_40$noise1 = sample(train_vars_noise_w_Ecig_40[[colnames(train_vars_noise_w_Ecig_40[3])]], 
                                           replace = TRUE) 
train_vars_noise_w_Ecig_40$noise2 = sample(train_vars_noise_w_Ecig_40[[colnames(train_vars_noise_w_Ecig_40[4])]], 
                                           replace = TRUE)
train_vars_noise_w_Ecig_40$noise3 = sample(train_vars_noise_w_Ecig_40[[colnames(train_vars_noise_w_Ecig_40[5])]], 
                                           replace = TRUE)
train_vars_noise_w_Ecig_40$noise4 = sample(train_vars_noise_w_Ecig_40[[colnames(train_vars_noise_w_Ecig_40[6])]], 
                                           replace = TRUE)
train_vars_noise_w_Ecig_40$noise5 = sample(train_vars_noise_w_Ecig_40[[colnames(train_vars_noise_w_Ecig_40[7])]], 
                                           replace = TRUE)

train_vars_noise_wo_Ecig_40$noise1 = sample(train_vars_noise_wo_Ecig_40[[colnames(train_vars_noise_wo_Ecig_40[3])]], 
                                           replace = TRUE) 
train_vars_noise_wo_Ecig_40$noise2 = sample(train_vars_noise_wo_Ecig_40[[colnames(train_vars_noise_wo_Ecig_40[4])]], 
                                           replace = TRUE)
train_vars_noise_wo_Ecig_40$noise3 = sample(train_vars_noise_wo_Ecig_40[[colnames(train_vars_noise_wo_Ecig_40[5])]], 
                                           replace = TRUE)
train_vars_noise_wo_Ecig_40$noise4 = sample(train_vars_noise_wo_Ecig_40[[colnames(train_vars_noise_wo_Ecig_40[6])]], 
                                           replace = TRUE)
train_vars_noise_wo_Ecig_40$noise5 = sample(train_vars_noise_wo_Ecig_40[[colnames(train_vars_noise_wo_Ecig_40[7])]], 
                                           replace = TRUE)

head(train_vars_noise_w_Ecig_40)

Smoking_Status,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,⋯,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976,noise1,noise2,noise3,noise4,noise5
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,⋯,19.06203,19.13556,16.93182,17.55702,17.05593,22.88241,20.68832,26.26236,17.50820,17.81758
0,21.08622,22.58769,21.05378,17.93584,17.33098,20.41479,20.37485,21.12876,21.01488,⋯,18.06379,19.39103,16.72112,19.27343,19.30881,23.00581,22.14102,26.95046,17.88701,17.69319
0,13.12099,14.44659,13.46827,16.51009,14.93820,15.66299,15.72731,13.78698,15.18975,⋯,19.49121,17.09470,15.56214,19.55038,19.54912,23.46020,22.77595,18.47581,21.08991,21.33890
0,19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,⋯,19.19085,19.41757,16.88993,19.35678,18.95550,22.38033,19.94664,18.18218,20.96841,22.54351
0,22.53459,24.36662,22.77595,16.69956,16.38411,22.55912,17.30008,22.80146,22.09338,⋯,16.32598,15.62938,17.37207,15.84334,17.82000,17.40963,13.46827,20.21918,26.11076,21.71412
0,21.50216,23.34553,21.53633,17.93372,24.26751,21.71412,21.32122,21.66214,21.71152,⋯,16.71579,18.52501,17.61326,17.74147,16.76017,23.90655,21.58279,17.32305,16.42013,20.05850


In [13]:
#calling rf fn
noise_rf_values_w_Ecig_40 = rf_classification(train_vars_noise_w_Ecig_40, "Smoking_Status", "pred_Smoking_Status")
noise_rf_values_wo_Ecig_40 = rf_classification(train_vars_noise_wo_Ecig_40, "Smoking_Status", 
                                               "pred_Smoking_Status")

In [12]:
#viewing results
noise_rf_confusion_matrix = data.frame(Model = c(rep(c("RF w/ Noise"), times = 2)), 
                                       Dataset = c("Ecig_40", "no_Ecig_40"), Kernel = c(rep(c(NA), times = 2)),
                                       rbind(noise_rf_values_w_Ecig_40[[2]], noise_rf_values_wo_Ecig_40[[2]]))
noise_rf_confusion_matrix

#filtering for most significant features
head(noise_rf_values_w_Ecig_40[[3]])
head(noise_rf_values_wo_Ecig_40[[3]]) 

Model,Dataset,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF w/ Noise,Ecig_40,NA,0.8866667,0.8933333,0.88,0.9250000,0.8866667
RF w/ Noise,no_Ecig_40,NA,0.8750000,0.8800000,0.87,0.9028571,0.8750000


MF,MeanDecreaseGini
<chr>,<dbl>
MF_97,7.05424062
MF_5,4.68378298
MF_292,0.27705811
MF_3345,0.24800949
MF_1713,0.18760701
MF_4803,0.16542196
MF_7423,0.15562403
MF_5782,0.15079241
MF_7221,0.14144676


MF,MeanDecreaseGini
<chr>,<dbl>
MF_97,7.05972731
MF_5,4.08153303
MF_13899,0.29520675
MF_4858,0.19183248
MF_5547,0.19155348
MF_292,0.18847797
MF_17380,0.15940730
MF_108,0.15200375
MF_873,0.15093885


# Support Vector Machine
- supervised learning models that can predict continuous (regression) or grouped/dichotomous (classification) outcomes
- predictors by projecting them onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)
- does a better job at handling a large number of predictors since p > n

In [40]:
#model errors are calculated to assess the predictive accuracy of the model
svm_classification = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, elsilon_values){
  
    #setting seed for reproducibility
    set.seed(12)
    
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    metrics = data.frame()
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        #now pruning parameters (based on the training dataset to prevent overfitting)
        svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = data_train, kernel = model, 
                       ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

        #choosing best model
        best_svm <- svr_tune$best.model

        #predicting with tuned parameters 
        data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                 positive = "1")
 
        #calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        #extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
   
        #adding values to df
        metrics = rbind(metrics, matrix_values)
  }
  
  #taking averages/sd 
  metrics = metrics %>%
    summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
  
  return(metrics)
}

In [ ]:
#calling fn
#linear kernel
svm_linear_w_Ecig_40 = svm_classification(molecular_feature_df_w_Ecig_40, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))
svm_linear_wo_Ecig_40 = svm_classification(molecular_feature_df_wo_Ecig_40, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))
#radial
svm_radial_w_Ecig_40 = svm_classification(molecular_feature_df_w_Ecig_40, "radial", "Smoking_Status", "pred_Smoking_Status",
                                             0.035, 1:5, seq(0,1,0.2))
svm_radial_wo_Ecig_40 = svm_classification(molecular_feature_df_wo_Ecig_40, "radial", "Smoking_Status", "pred_Smoking_Status",
                                             0.035, 1:5, seq(0,1,0.2))
#polynomial
#elison value is fixed at 0.1, but I left it so I didn't have to create a new function
svm_polynomial_w_Ecig_40 = svm_classification(molecular_feature_df_w_Ecig_40, "polynomial", "Smoking_Status", 
                                             "pred_Smoking_Status", 0.035, 1:5, 0.1)
svm_polynomial_wo_Ecig_40 = svm_classification(molecular_feature_df_wo_Ecig_40, "polynomial", "Smoking_Status", 
                                  "pred_Smoking_Status", 0.035, 1:5, 0.1)

In [13]:
#creating one combined df
svm_final_df = data.frame(rbind(svm_linear_w_Ecig_40, svm_radial_w_Ecig_40, svm_polynomial_w_Ecig_40,
                               svm_linear_wo_Ecig_40, svm_radial_wo_Ecig_40, svm_polynomial_wo_Ecig_40))

svm_final_df = data.frame(Model = c(rep(c("Original SVM"), times = 6)), Dataset = c(rep(c("Ecig_40"), times = 3), 
                            rep(c("no_Ecig_40"), times = 3)), Kernel = rep(c("linear", "radial", "polynomial"), 
                                                                           times = 2), svm_final_df)
svm_final_df

Dataset,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ecig_40,linear,0.7614286,0.8095238,0.7133333,0.8044444,0.7614286
Ecig_40,radial,0.5000000,1.0000000,0.0000000,0.5424242,0.5000000
Ecig_40,polynomial,0.6057143,0.9047619,0.3066667,0.6187013,0.6057143
no_Ecig_40,linear,0.8100000,0.8000000,0.8200000,0.8666667,0.8100000
no_Ecig_40,radial,0.5000000,1.0000000,0.0000000,0.5348485,0.5000000
no_Ecig_40,polynomial,0.6057143,0.9047619,0.3066667,0.6070996,0.6057143


> The linear kernel performed the best overall and the inclusion of the outlier didn't make much of a difference.

# Multicollinearity Variable Selection w/ SVM

- Determing if there are highly correlated variables that can be removed, because multicollinearity makes variables that should be significant appear insignificant

In [3]:
#creating correlation matrix
#the cut off can be arbitrary, but due to the high number of correlations, 0.85 became the cutoff
corr_matrix_w_Ecig_40 = rcorr(as.matrix(molecular_feature_df_w_Ecig_40[2:3873]), type = "spearman")
#creating df where I can easily view coefficents that are highly correlated (rho > 0.7 or rho < -0.7)
corr_matrix_df_w_Ecig_40 = data.frame(corr_matrix_w_Ecig_40$r) %>%
  rownames_to_column(var = "Variable1") %>%
  #needed to specificy package of melt for it to work
  reshape2::melt(variable = "Variable2", value.name = "Spearman Coefficient") %>%
  #also removed values = 1, because those were correlations between the same variable
  filter(abs(`Spearman Coefficient`) < 1 & abs(`Spearman Coefficient`) > 0.7) %>%
  arrange(desc(`Spearman Coefficient`)) %>%
  #removing duplicate spearman coefficient values and keeping all columns
  distinct(`Spearman Coefficient`, .keep_all = TRUE) 

corr_matrix_wo_Ecig_40 = rcorr(as.matrix(molecular_feature_df_wo_Ecig_40[2:3873]), type = "spearman")
#creating df where I can easily view coefficents that are highly correlated (rho > 0.7 or rho < -0.7)
corr_matrix_df_wo_Ecig_40 = data.frame(corr_matrix_wo_Ecig_40$r) %>%
  rownames_to_column(var = "Variable1") %>%
  #needed to specificy package of melt for it to work
  reshape2::melt(variable = "Variable2", value.name = "Spearman Coefficient") %>%
  #also removed values = 1, because those were correlations between the same variable
  filter(abs(`Spearman Coefficient`) < 1 & abs(`Spearman Coefficient`) > 0.7) %>%
  arrange(desc(`Spearman Coefficient`)) %>%
  #removing duplicate spearman coefficient values and keeping all columns
  distinct(`Spearman Coefficient`, .keep_all = TRUE) 

head(corr_matrix_df_w_Ecig_40)

Using Variable1 as id variables

Using Variable1 as id variables



,Variable1,Variable2,Spearman Coefficient
,<chr>,<fct>,<dbl>
1,MF_586,MF_7,0.9987726
2,MF_124,MF_34,0.9984220
3,MF_516,MF_7,0.9974284
4,MF_301,MF_133,0.9973115
5,MF_54,MF_4,0.9969024
6,MF_2762,MF_974,0.9963764


In [4]:
corr_matrix_df_w_Ecig_40
corr_matrix_df_wo_Ecig_40

Variable1,Variable2,Spearman Coefficient
<chr>,<fct>,<dbl>
MF_586,MF_7,0.9987726
MF_124,MF_34,0.9984220
MF_516,MF_7,0.9974284
MF_301,MF_133,0.9973115
MF_54,MF_4,0.9969024
MF_2762,MF_974,0.9963764
MF_586,MF_516,0.9960257
MF_3375,MF_301,0.9945061
MF_2827,MF_277,0.9942139


Variable1,Variable2,Spearman Coefficient
<chr>,<fct>,<dbl>
MF_586,MF_7,0.9987696
MF_124,MF_34,0.9986465
MF_516,MF_7,0.9973546
MF_301,MF_133,0.9971700
MF_54,MF_4,0.9969239
MF_2762,MF_974,0.9963702
MF_586,MF_516,0.9958165
MF_1409,MF_1136,0.9945246
MF_3375,MF_301,0.9941555


In [8]:
#determining how many times each feature occurs
correlation_occurences1_w_Ecig_40 = corr_matrix_df_w_Ecig_40 %>%
    group_by(Variable1) %>%
    count() %>%
    arrange(-n) %>%
    filter(n > 20)

#correlation_occurences1

correlation_occurences2_w_Ecig_40 = corr_matrix_df_w_Ecig_40 %>%
    group_by(Variable2) %>%
    count() %>%
    arrange(-n) %>%
    filter(n > 20)

#correlation_occurences2

all_corr_occurences = unique(c(correlation_occurences1$Variable1, as.character(correlation_occurences2$Variable2)))

In [9]:
all_corr_occurences

character(0)

In [10]:
#removing molecular features (MF) that correlate most often
`%notin%` <- Negate(`%in%`)
penalized_dataset = molecular_feature_df[,colnames(molecular_feature_df) %notin% all_corr_occurences]

In [13]:
#calling fn
#linear kernel
svm_linear = svm_classification(penalized_dataset, "linear", "Smoking_Status", "pred_Smoking_Status",
                                             0.035, 1:5, seq(0,1,0.2))
#radial
svm_radial = svm_classification(penalized_dataset, "radial", "Smoking_Status", "pred_Smoking_Status",
                                             0.035, 1:5, seq(0,1,0.2))
#polynomial
#elison value is fixed at 0.1, but I left it so I didn't have to create a new function
svm_polynomial = svm_classification(penalized_dataset, "polynomial", "Smoking_Status", 
                                             "pred_Smoking_Status", 0.035, 1:5, 0.1)


#creating one combined df
svm_penalized_final_df = data.frame(rbind(svm_linear, svm_radial, svm_polynomial))
svm_penalized_final_df = data.frame(Kernel = c("linear", "radial", "polynomial"), svm_penalized_final_df)
svm_penalized_final_df

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls > cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls > cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls > cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
linear,0.4804762,0.5476190,0.4133333,0.5018254,0.5766667
radial,0.5252381,0.4904762,0.5600000,0.6000000,0.5919048
polynomial,0.4528571,0.4523810,0.4533333,NaN,0.4528571


# RF Variable Selection w/ SVM

Calculating the average noise rank to determine, which predictors ranked higher than the average noise rank. Then filtering the original datasets for those predictors and rerunning SVM.

In [14]:
#calculating avg noise rank out of all variables
noise_importance_values_w_Ecig_40 = noise_rf_values_w_Ecig_40[[3]] %>%
    column_to_rownames(var = "MF")
noise_importance_values_wo_Ecig_40 = noise_rf_values_wo_Ecig_40[[3]] %>%
    column_to_rownames(var = "MF")

noise1rank = which(rownames(noise_importance_values_w_Ecig_40) == "noise1")/nrow(noise_importance_values_w_Ecig_40)
noise2rank = which(rownames(noise_importance_values_w_Ecig_40) == "noise2")/nrow(noise_importance_values_w_Ecig_40)
noise3rank = which(rownames(noise_importance_values_w_Ecig_40) == "noise3")/nrow(noise_importance_values_w_Ecig_40)
noise4rank = which(rownames(noise_importance_values_w_Ecig_40) == "noise4")/nrow(noise_importance_values_w_Ecig_40)
noise5rank = which(rownames(noise_importance_values_w_Ecig_40) == "noise5")/nrow(noise_importance_values_w_Ecig_40)

#calculating avg mean decrease gini or background noise importance value
avg_noise_rank_w_Ecig_40 = (noise1rank + noise2rank + noise3rank + noise4rank + noise5rank)/5
avg_noise_rank_w_Ecig_40

noise1rank = which(rownames(noise_importance_values_wo_Ecig_40) == "noise1")/nrow(noise_importance_values_wo_Ecig_40)
noise2rank = which(rownames(noise_importance_values_wo_Ecig_40) == "noise2")/nrow(noise_importance_values_wo_Ecig_40)
noise3rank = which(rownames(noise_importance_values_wo_Ecig_40) == "noise3")/nrow(noise_importance_values_wo_Ecig_40)
noise4rank = which(rownames(noise_importance_values_wo_Ecig_40) == "noise4")/nrow(noise_importance_values_wo_Ecig_40)
noise5rank = which(rownames(noise_importance_values_wo_Ecig_40) == "noise5")/nrow(noise_importance_values_wo_Ecig_40)

avg_noise_rank_wo_Ecig_40 = (noise1rank + noise2rank + noise3rank + noise4rank + noise5rank)/5
avg_noise_rank_wo_Ecig_40

[1] 0.5411916

[1] 0.8350271

In [35]:
for (i in 1:length(noise_importance_values_w_Ecig_40$MeanDecreaseGini)){
    #if the row number (index)/# of variables is less than the avg noise rank the loop will continue
    if (i/length(noise_importance_values_w_Ecig_40$MeanDecreaseGini) > avg_noise_rank_w_Ecig_40){
        last_significant_rank_w_Ecig_40 = i - 1
        break
    }
}
last_significant_rank_w_Ecig_40

for (i in 1:length(noise_importance_values_wo_Ecig_40$MeanDecreaseGini)){
    #if the row number (index)/# of variables is less than the avg noise rank the loop will continue
    if (i/length(noise_importance_values_wo_Ecig_40$MeanDecreaseGini) > avg_noise_rank_wo_Ecig_40){
        last_significant_rank_wo_Ecig_40 = i - 1
        break
    }
}
last_significant_rank_wo_Ecig_40

[1] 2098

[1] 3237

Therefore, the dataset containing Ecig_40 contains 2,098 MFs above background, while the dataset without the outlier contains 3,237. This means that the outlier dampened the significance of about 1,000 MF just by being included in the dataset. 

In [50]:
#now getting those significant MFs
significant_MFs_w_Ecig_40 = rownames(noise_importance_values_w_Ecig_40)[1:last_significant_rank_w_Ecig_40]
significant_MFs_wo_Ecig_40 = rownames(noise_importance_values_wo_Ecig_40)[1:last_significant_rank_wo_Ecig_40]

#now subsetting the original dfs
rf_subset_MF_df_w_Ecig_40 = cbind(Smoking_Status = molecular_feature_df_w_Ecig_40$Smoking_Status, molecular_feature_df_w_Ecig_40[,colnames(molecular_feature_df_w_Ecig_40) %in% significant_MFs_w_Ecig_40])
rf_subset_MF_df_wo_Ecig_40 = cbind(Smoking_Status = molecular_feature_df_wo_Ecig_40$Smoking_Status, molecular_feature_df_wo_Ecig_40[,colnames(molecular_feature_df_wo_Ecig_40) %in% significant_MFs_wo_Ecig_40])
head(rf_subset_MF_df_w_Ecig_40)

,Smoking_Status,MF_5,MF_11,MF_12,MF_13,MF_14,MF_15,MF_17,MF_18,MF_21,⋯,MF_53619,MF_54327,MF_54613,MF_56870,MF_57989,MF_61492,MF_62137,MF_62282,MF_64936,MF_78976
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,18.02809,18.41799,19.31361,19.46777,20.80616,18.72405,20.11458,18.49551,18.42766,⋯,18.26770,16.83157,17.55214,19.09192,19.07769,18.88772,17.47760,18.83525,17.09794,17.05593
2,0,17.93584,20.37485,21.12876,21.01488,19.71124,19.17696,19.82979,19.80685,22.61087,⋯,19.82107,16.89255,19.40468,19.14730,17.29091,15.81175,16.35027,19.31905,19.06421,19.30881
3,0,16.51009,15.72731,13.78698,15.18975,18.19551,18.84322,18.65454,17.52149,18.26920,⋯,18.97545,19.18696,19.22546,18.95185,19.30798,18.98001,15.64893,16.80361,16.97302,19.54912
4,0,16.22547,21.14531,22.07857,21.97502,17.30567,19.61857,18.35221,17.14531,22.47085,⋯,19.31315,18.69526,18.74873,19.40897,19.53008,17.50812,17.78108,19.06662,19.47635,19.32788
5,0,16.73360,17.64979,19.64269,19.32233,21.33752,19.56012,20.23552,18.70226,21.53982,⋯,19.69692,19.06106,19.32826,19.12974,19.11060,17.61151,17.97428,19.13998,18.50336,18.95550
6,0,16.69956,17.30008,22.80146,22.09338,19.51343,19.39161,20.72501,21.45596,21.96066,⋯,18.53569,17.74897,18.41935,19.60590,13.81876,19.24975,16.06782,19.19848,17.46699,17.82000


In [52]:
#calling fn
#linear kernel
subset_svm_linear_w_Ecig_40 = svm_classification(rf_subset_MF_df_w_Ecig_40, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))
subset_svm_linear_wo_Ecig_40 = svm_classification(rf_subset_MF_df_wo_Ecig_40, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))
#radial
subset_svm_radial_w_Ecig_40 = svm_classification(rf_subset_MF_df_w_Ecig_40, "radial", "Smoking_Status", "pred_Smoking_Status",
                                             0.035, 1:5, seq(0,1,0.2))
subset_svm_radial_wo_Ecig_40 = svm_classification(rf_subset_MF_df_wo_Ecig_40, "radial", "Smoking_Status", "pred_Smoking_Status",
                                             0.035, 1:5, seq(0,1,0.2))
#polynomial
subset_svm_polynomial_w_Ecig_40 = svm_classification(rf_subset_MF_df_w_Ecig_40, "polynomial", "Smoking_Status", 
                                             "pred_Smoking_Status", 0.035, 1:5, 0.1)
subset_svm_polynomial_wo_Ecig_40 = svm_classification(rf_subset_MF_df_wo_Ecig_40, "polynomial", "Smoking_Status", 
                                  "pred_Smoking_Status", 0.035, 1:5, 0.1)

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [53]:
#creating one combined df
subset_svm_final_df = data.frame(rbind(subset_svm_linear_w_Ecig_40, subset_svm_radial_w_Ecig_40, 
                                       subset_svm_polynomial_w_Ecig_40,subset_svm_linear_wo_Ecig_40, 
                                       subset_svm_radial_wo_Ecig_40, subset_svm_polynomial_wo_Ecig_40))

subset_svm_final_df = data.frame(Model = c(rep(c("SVM w/ RF Var Selection"), times = 6)), 
                                 Dataset = c(rep(c("Ecig_40"), times = 3), rep(c("no_Ecig_40"), times = 3)), 
                                 Kernel = rep(c("linear", "radial", "polynomial"), times = 2), 
                                 subset_svm_final_df)
subset_svm_final_df

Model,Dataset,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SVM w/ RF Var Selection,Ecig_40,linear,0.7614286,0.8095238,0.7133333,0.8044444,0.7614286
SVM w/ RF Var Selection,Ecig_40,radial,0.5000000,1.0000000,0.0000000,0.5424242,0.5000000
SVM w/ RF Var Selection,Ecig_40,polynomial,0.6304762,0.8476190,0.4133333,0.6806061,0.6304762
SVM w/ RF Var Selection,no_Ecig_40,linear,0.7957143,0.8047619,0.7866667,0.8230952,0.7957143
SVM w/ RF Var Selection,no_Ecig_40,radial,0.5000000,1.0000000,0.0000000,0.5348485,0.5000000
SVM w/ RF Var Selection,no_Ecig_40,polynomial,0.6557143,0.9047619,0.4066667,0.6461905,0.6557143


In [ ]:
#all results
final_df = rbind(rf_confusion_matrix, svm_final_df, svm_penalized_final_df, subset_svm_final_df)
final_df